In [1]:
import json
with open('/Users/Ernesto/.secret/yelp_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)
login.keys()


dict_keys(['client-id', 'api-key'])

In [2]:
!pip install yelpapi

In [3]:
from yelpapi import YelpAPI
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [4]:
# Quick Test Query
results = yelp_api.search_query(location='Baltimore, MD',
                                       term='Crab Cake')
print(type(results))
results.keys()


<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [5]:
help(yelp_api.search_query)



Help on method search_query in module yelpapi.yelpapi:

search_query(**kwargs) method of yelpapi.yelpapi.YelpAPI instance
    Query the Yelp Search API.
    
    documentation: https://www.yelp.com/developers/documentation/v3/business_search
    
    required parameters:
        * one of either:
            * location - text specifying a location to search for
            * latitude and longitude



In [6]:
# use our yelp_api variable's search_query method to perform our API call
search_results = yelp_api.search_query(location='NY, NY',
                                       term='Pizza')
print(type(search_results))
search_results.keys()



<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

The package returns the results in the JSON format we have been exploring. Note that the exact results may vary as the Yelp site is constantly changing

In [7]:
search_results['total']



12400

In [9]:
import pandas as pd
biz = pd.DataFrame(search_results['businesses'])
biz.head(2) 




,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,zj8Lq1T8KIC5zwFief15jg,prince-street-pizza-new-york-2,Prince Street Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/I4gm7i...,False,https://www.yelp.com/biz/prince-street-pizza-n...,4660,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.72308755605564, 'longitude': -...","[pickup, delivery]",$,"{'address1': '27 Prince St', 'address2': None,...",+12129664100,(212) 966-4100,1961.877142
1,ysqgdbSrezXgVwER2kQWKA,julianas-brooklyn-3,Juliana's,https://s3-media2.fl.yelpcdn.com/bphoto/NVoLFl...,False,https://www.yelp.com/biz/julianas-brooklyn-3?a...,2584,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.70274718768062, 'longitude': -...",[delivery],$$,"{'address1': '19 Old Fulton St', 'address2': '...",+17185966700,(718) 596-6700,308.569844


In [10]:
## total number of matching businesses
search_results['total']



12400

In [12]:
## how many businesses in our results
len(search_results['businesses'])




20

This is what Yelp Fusion is doing as well! The general term for this is "Pagination".



The Yelp API will only return a "page" of 20 results at a time.

If we want to get the next page of results, we will perform another API call, but we will add an additional argument called "offset".

The offset is what # result to use as the FIRST result for the page.

If we had 20 businesses in our first result, we would want to add an offset of 20.

In [13]:
# add offset to our original api call
search_results = yelp_api.search_query(location='NY, NY',
                                       term='Pizza',
                                       offset = 20)



In [14]:
biz20 = pd.DataFrame(search_results['businesses'])
biz20.head(2)



,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,6ECcDYsRgmfv0eKXk-c11Q,la-nonna-krispy-krust-pizza-brooklyn,La Nonna Krispy Krust Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/fVNfYk...,False,https://www.yelp.com/biz/la-nonna-krispy-krust...,73,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.717953, 'longitude': -73.9637027}","[delivery, pickup]",$$,"{'address1': '175 Kent Ave', 'address2': None,...",+13473842100,(347) 384-2100,2929.556415
1,CiRj1B-gK5tRfbMkq3SBog,robertas-new-york-3,Roberta's,https://s3-media3.fl.yelpcdn.com/bphoto/__bI5t...,False,https://www.yelp.com/biz/robertas-new-york-3?a...,4,"[{'alias': 'pizza', 'title': 'Pizza'}]",5.0,"{'latitude': 40.71995520720623, 'longitude': -...","[delivery, pickup]",NaN,"{'address1': '196 Stanton St', 'address2': '',...",,,1871.408970


Now you can combine the results (so far) into one dataframe using pd.concat()

In [15]:
## concatenate the previous results and new results. 
businesses = pd.concat([biz, biz20],
                      ignore_index=True)
display(businesses.head(3), businesses.tail(3))



,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,zj8Lq1T8KIC5zwFief15jg,prince-street-pizza-new-york-2,Prince Street Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/I4gm7i...,False,https://www.yelp.com/biz/prince-street-pizza-n...,4660,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.72308755605564, 'longitude': -...","[pickup, delivery]",$,"{'address1': '27 Prince St', 'address2': None,...",+12129664100,(212) 966-4100,1961.877142
1,ysqgdbSrezXgVwER2kQWKA,julianas-brooklyn-3,Juliana's,https://s3-media2.fl.yelpcdn.com/bphoto/NVoLFl...,False,https://www.yelp.com/biz/julianas-brooklyn-3?a...,2584,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.70274718768062, 'longitude': -...",[delivery],$$,"{'address1': '19 Old Fulton St', 'address2': '...",+17185966700,(718) 596-6700,308.569844
2,v1DHGRNCH9247WLYoaoA9A,l-industrie-pizzeria-brooklyn,L'industrie Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/SkMxUE...,False,https://www.yelp.com/biz/l-industrie-pizzeria-...,837,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.71162, 'longitude': -73.95783}",[delivery],$,"{'address1': '254 S 2nd St', 'address2': '', '...",+17185990002,(718) 599-0002,3145.016041


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
37,Aazjxg21VH2cncWYBAv68Q,sottocasa-pizzeria-brooklyn-3,Sottocasa Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/MbBAMb...,False,https://www.yelp.com/biz/sottocasa-pizzeria-br...,699,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.688285, 'longitude': -73.989006}",[delivery],$$,"{'address1': '298 Atlantic Ave', 'address2': N...",+17188528758,(718) 852-8758,1955.310589
38,RfFXF-Momn0tLIgh1TZKLw,di-fara-pizza-new-york-2,Di Fara Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/SaWdpz...,False,https://www.yelp.com/biz/di-fara-pizza-new-yor...,28,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.706894091267536, 'longitude': ...","[delivery, pickup]",NaN,"{'address1': '108 South St', 'address2': '', '...",+19173883130,(917) 388-3130,657.021625
39,affWh2WFSMayL04l9lr47g,roberta-s-brooklyn,Roberta’s,https://s3-media1.fl.yelpcdn.com/bphoto/rdK-so...,False,https://www.yelp.com/biz/roberta-s-brooklyn?ad...,11,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.71613781459604, 'longitude': -...",[],NaN,"{'address1': '6 Grand St', 'address2': '', 'ad...",,,2620.492306


In [17]:
len(businesses)

40